In [16]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [17]:
data= pd.read_csv('bnpl_scaled_cleaned.csv')

In [18]:
def binary_col(data):
    #return the binary columns
    binary_cols=[col for col in data.columns if data[col].nunique() == 2]
    return binary_cols

In [19]:
def continuous_col(data):
    #return the continuous columns
    continuous_cols=[col for col in data.columns if data[col].nunique() > 2]
    return continuous_cols
    

In [20]:
def Scaler(data):
    binary_cols = binary_col(data)
    continuous_cols = continuous_col(data)
    scaler = StandardScaler()
   # Scale the continuous columns
    scaled_continuous = scaler.fit_transform(data[continuous_cols])
    scaled_continuous_df = pd.DataFrame(scaled_continuous, columns=continuous_cols)
    # Combine the scaled continuous columns with the binary columns
    result = pd.concat([data[binary_cols].reset_index(drop=True), scaled_continuous_df.reset_index(drop=True)], axis=1)
    return result

In [21]:
data_scaled = Scaler(data)

In [24]:
data_scaled_cleaned = pd.read_csv('bnpl_scaled_cleaned.csv')

In [ ]:
# concat the following stress_usage_interaction = financial_stress_score × bnpl_usage_frequency
data_scaled_cleaned['stress_usage_interaction'] = (
    data_scaled_cleaned['financial_stress_score'] * data_scaled_cleaned['bnpl_usage_frequency']
)
# debt_per_delinquency = bnpl_debt_ratio / (payment_delinquency_count + 1)  
data_scaled_cleaned['debt_per_delinquency'] = (
    data_scaled_cleaned['bnpl_debt_ratio'] / (data_scaled_cleaned['payment_delinquency_count'] + 1)
)
#avg_debt_per_txn = bnpl_debt_ratio / (bnpl_usage_frequency + 1)
data_scaled_cleaned['avg_debt_per_txn'] = (
    data_scaled_cleaned['bnpl_debt_ratio'] / (data_scaled_cleaned['bnpl_usage_frequency'] + 1)
)

#adjusted_debt_interaction = bnpl_debt_ratio * financial_stress_score
data_scaled_cleaned['adjusted_debt_interaction'] = (
    data_scaled_cleaned['bnpl_debt_ratio'] + data_scaled_cleaned['financial_stress_score']
)
data_scaled_cleaned

In [ ]:
#scale the new features
new_features = ['stress_usage_interaction', 'debt_per_delinquency', 'avg_debt_per_txn', 'adjusted_debt_interaction']
data_scaled_cleaned[new_features] = Scaler(data_scaled_cleaned[new_features])
data_scaled_cleaned